In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENROUTER_API_KEY")
print("KEY:", key)
print("Starts with sk-or-:", key.startswith("sk-or-") if key else None)


KEY: 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
Starts with sk-or-: True


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(override=True)
from datetime import datetime
import json
import uuid

##############################################################################
def store_agent_output(
    prompt: str,
    output: str,
    model_name: str,
    path: str = "agent_runs.jsonl",
    extra_metadata: dict | None = None
):
    record = {
        "run_id": str(uuid.uuid4()),
        "prompt": prompt,
        "final_output": output,
        "model": model_name,
        "timestamp": datetime.utcnow().isoformat(),
        "metadata": extra_metadata or {}
    }

    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")
#############################################################################3

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)

# First API call with reasoning
response = client.chat.completions.create(
  model="nvidia/nemotron-3-nano-30b-a3b:free",
  messages=[
          {
            "role": "user",
            "content": "pick a business area that might be worth exploring for an Agentic AI combined with RAG system opportunity."
          }
        ],
  extra_body={"reasoning": {"enabled": True}}
)

# Extract the assistant message with reasoning_details
response = response.choices[0].message
print("First response:", response.content)

########################################################################################################
# Preserve the assistant message with reasoning_details
messages = [
  {"role": "user", "content": "pick a business area that might be worth exploring for an Agentic AI combined with RAG system opportunity."},
  {
    "role": "assistant",
    "content": response.content,
    "reasoning_details": response.reasoning_details  # Pass back unmodified
  },
  {"role": "user", "content": "Make sure it generates good amount of revenue."}
]

# Second API call - model continues reasoning from where it left off
response2 = client.chat.completions.create(
  model="nvidia/nemotron-3-nano-30b-a3b:free",
  messages=messages,
  extra_body={"reasoning": {"enabled": True}}
)

#########################################################################################################

messages = [
  {"role": "user", "content": "pick a business area that might be worth exploring for an Agentic AI combined with RAG system opportunity."},
  {
    "role": "assistant",
    "content": response.content,
    "reasoning_details": response.reasoning_details  # Pass back unmodified
  },
  {"role": "user", "content": "Make sure it generates good amount of revenue."},
  {
    "role": "assistant",
    "content": response2.choices[0].message.content,
    "reasoning_details": response2.choices[0].message.reasoning_details  # Pass back unmodified
  },
  {"role": "user", "content": "Now propose the agentic AI solution."}
]
# Third API call - model continues reasoning from where it left off
response3 = client.chat.completions.create(
  model="nvidia/nemotron-3-nano-30b-a3b:free",
  messages=messages,
  extra_body={"reasoning": {"enabled": True}}
)

First response: **Business Area to Explore:** **Enterprise Knowledge‑Centred Decision‑Support (e.g., legal, compliance, and risk advisory for large corporations)**  

---

### Why This Area is a Sweet Spot for an **Agentic AI + RAG** System

| Dimension | Value Proposition |
|-----------|-------------------|
| **Complex, High‑Stakes Knowledge** | Large enterprises generate massive volumes of internal policies, case law, regulatory filings, audit reports, and proprietary research. Decision‑makers need *accurate, up‑to‑date* answers anchored in that corpus to mitigate legal risk, satisfy audits, or advise on compliance. |
| **Dynamic, Multi‑Step Reasoning** | A single query often leads to a chain of dependent steps: e.g., “What GDPR‑related obligations arise when we launch a new SaaS product in the EU?” → Identify relevant regulation → Locate recent rulings → Map internal policy → Draft compliance checklist → Simulate regulator‑style Q&A. Agentic AI can orchestrate these steps autonomous